In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json("../data/민원(콜센터) 질의응답_다산콜센터_일반행정 문의_Training.json")

In [ ]:
df.info()

In [ ]:
# 결측치의 개수를 확인 
df.isna().sum()

In [ ]:
df.head(10)

### 문제 
1. 일반행정 데이터와, 대중 교통 데이터를 로드 
2. 두개의 데이터프레임을 결합 (단순한 행 결합)
3. 데이터의 필터링 고객질문에 대한 상담사의 답변이 즉각적으로 오는 데이터들만 필터 
4. 질문 중 중복 데이터를 제거 
5. 질문들을 모아서 토큰화, 백터화 
6. 그 외의 질문 목록을 이용하여 코사인 유사도 확인하고 유사 질문과 답변을 출력 

In [ ]:
# 2개의 데이터프레임을 로드 
df = pd.read_json("../data/민원(콜센터) 질의응답_다산콜센터_대중교통 안내_Training.json")
df2 = pd.read_json("../data/민원(콜센터) 질의응답_다산콜센터_일반행정 문의_Training.json")


In [ ]:
# 2개의 데이터프레임을 단순한 행 결합 (union 결합)
total_df = pd.concat( [df, df2], ignore_index=True  )

In [ ]:
# 단순 결합시 주의할점 : 인덱스의 값이 중복 값이 생기는 부분 
total_df.loc[0, ]

In [ ]:
total_df.head()

In [ ]:
# 고객질문(요청) 컬럼의 데이터들의 개수를 확인 
total_df['고객질문(요청)'].value_counts()
#공백의 데이터가 여러개가 존재 확인 

In [ ]:
# 공백으로 이루어진 value들을 통일화
# 스리즈에서 각각의 value을 추출하여 함수에 대입 -> map()
total_df = total_df.map(
    lambda x : str(x).strip() 
)

In [ ]:
total_df.head()

In [ ]:
# 1번 조건식 -> 현재 행에서 고객질문(요청) 데이터가 ''가 아니고 
#               다음 행의 상담사답변의 value가 ''이 아닌경우
flag1 = (total_df['고객질문(요청)'] != '') & \
    (total_df['상담사답변'].shift(-1) != '')

In [ ]:
# 2번 조건식 -> 현재 행에서 상담사답변이 '' 가 아니고
#               전 행의 고객질문(요청) 데이터가 '' 가 아닌 경우
flag2 = (total_df['상담사답변'] != '') & \
        (total_df['고객질문(요청)'].shift(1) != ''  )

In [ ]:
# flag1은 고객 질문만 나오고
# flag2는 상담사 답변 
# 둘 중 하나만 True라면 
total_df = total_df.loc[flag1 | flag2,]

In [58]:
# 고객 질문 데이터중 중복 데이터는 제거 
total_df['상담사답변'] = total_df['상담사답변'].shift(-1)

In [61]:
# 고객질문(요청) 데이터에서 ''가 아닌 데이터만 필터
total_df = total_df.loc[
    total_df['고객질문(요청)'] != ''
]

In [64]:
total_df = total_df.drop_duplicates('고객질문(요청)').reset_index(drop=True)

In [ ]:
total_df

In [67]:
# 토큰화, 백터화 정의 
from konlpy.tag import Komoran
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [68]:
komoran = Komoran()

def tokenize(text):
    return komoran.morphs(text)

vectorizer = TfidfVectorizer(
    tokenizer=tokenize, 
    lowercase=False, 
    ngram_range= (1,1), 
    min_df = 5, 
    max_df= 0.8
)

In [69]:
# total_df에서 고객질문(요청) 데이터를 토큰화, 백터화 작업 
X = vectorizer.fit_transform(
    total_df['고객질문(요청)'].values
)

c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [76]:
# 질문 목록 
new_questions = [
    '여권 재발급 신청 방법을 알려줘', 
    '전입 신고가 인터넷으로 가능한가요?',
    '지방세 환급금을 어디서 신청하나요?'
]

In [77]:
# new_questions도 토큰, 백터화 -> fit() x
test = vectorizer.transform(new_questions)

In [78]:
# 코사인 유사도 생성 
sims = cosine_similarity(test, X)

In [79]:
sims

array([[0.        , 0.        , 0.03641695, ..., 0.        , 0.        ,
        0.        ],
       [0.02427217, 0.        , 0.03673714, ..., 0.        , 0.        ,
        0.05143491],
       [0.02110636, 0.        , 0.03996141, ..., 0.03383102, 0.03217321,
        0.02175454]], shape=(3, 18952))

In [81]:
for idx, sim in enumerate(sims):
    question = new_questions[idx]
    print("유저의 질문 : ", question)
    # sim데이터에서 내림차순정렬을 한 인덱스의 목록 
    sim_idxs = sim.argsort()[::-1]
    for i in sim_idxs[:2]:
        # i : 유저의 질문에 가장 유사한 질문의 인덱스
        print(f"유사도 : {round(sim[i], 3)} \
              유사 질문 : {total_df.loc[i, '고객질문(요청)']}, \
              답변 : {total_df.loc[i, '상담사답변']}")

유저의 질문 :  여권 재발급 신청 방법을 알려줘
유사도 : 0.619               유사 질문 : 신청방법을 알려주세요.,               답변 : 주민등록상 세대주와 가까운 주민센터 또는 복지로 홈페이지에서 신청가능하세요.
유사도 : 0.568               유사 질문 : 신청방법 좀 알려주세요?,               답변 : 우선 사이트에 접속하셔서 회원가입을 해주세요. 청소년일경우 공인인증서가 없으면 본인확인절차를 거쳐 회원가입을 하고, 부모님이나 세대주분께서 가입을 하실 경우 공인인증서로 가입할 수 있습니다.
유저의 질문 :  전입 신고가 인터넷으로 가능한가요?
유사도 : 0.738               유사 질문 : 인터넷으로도 신고가능한가요?,               답변 : 방문 접수밖에 안됩니다.
유사도 : 0.682               유사 질문 : 인터넷으로 가능한가요?,               답변 : 인터넷으로 신청 가능합니다.
유저의 질문 :  지방세 환급금을 어디서 신청하나요?
유사도 : 0.76               유사 질문 : 지방세 환급금 신청은 어떻게 해야하죠?,               답변 : 인터넷에서 접수를 하셔야 합니다
유사도 : 0.566               유사 질문 : 환급금을 기부할 수도 있나요?,               답변 : 네 환급금을 사회복지공동모금회에 본인 명의로 기부가 가능합니다


### 문제 2
- 고객질문의 데이터를 이용하여 카테고리를 분류하는 모델을 생성 
    - 고객질문 데이터들을 이용하여 토큰화, 백터화 작업 (독립 변수) 
    - 카테고리 일반행정, 대중교통을 타켓 데이터 (종속 변수)
        - 카테고리 데이터를 LabelEncoder()를 이용하여 수치화 변환
    - SVC 모델을 이용하여 백터화된 데이터와 카테고리 데이터를 이용하여 학습 
    - new_qeustions의 카테고리를 확인 